## Introduction & Context

This notebook cleans the raw ATP matches data (2015–2024), handling missing values, parsing match scores, and exporting standardized datasets for modeling.

## Imports and Setup

Importing required Python libraries for data loading and cleaning.

In [1]:
import pandas as pd
import numpy as np

## Load Raw Data

Load the raw ATP matches data and perform initial transformations (date formatting, removing unused columns).

In [2]:
# CHANGES: File name to correct year
data = pd.read_csv("data/raw/atp_matches_2024.csv")

In [3]:
# Transform variable types
data["tourney_date"] = pd.to_datetime(data["tourney_date"], format="%Y%m%d")

In [4]:
# Drop match_num
data = data.drop(columns=['match_num'])

## Remove Irrelevant Matches

We exclude non-standard tour-level events that do not follow ATP tour structures.

In [5]:
def remove_tournament(data, column, value, label):
    print(f"Entries before removing {label}: {len(data)}")
    data = data.drop(data[data[column] == value].index)
    print(f"Entries after removing {label}: {len(data)}")
    return data

data = remove_tournament(data, "tourney_level", "D", "Davis Cup")
data = remove_tournament(data, "tourney_name", "Laver Cup", "Laver Cup")
data = remove_tournament(data, "tourney_name", "United Cup", "United Cup")


data.reset_index(inplace=True, drop=True)

Entries before removing Davis Cup: 3076
Entries after removing Davis Cup: 2816
Entries before removing Laver Cup: 2816
Entries after removing Laver Cup: 2807
Entries before removing United Cup: 2807
Entries after removing United Cup: 2782


## Fix Minutes Data

Some matches are missing or incorrectly recorded match duration (minutes). We identify them, manually correct via Excel, and merge fixed values back.

In [6]:
# Replacing minutes for walkover matches with NaN (rather than 0.0)
data.loc[
    (data["score"] == "W/O") & (data["minutes"] == 0.0),
    'minutes'
] = np.nan

In [7]:
# Remove matches that ended in default
data = data[~data["score"].str.contains('Def.', na=False)]

In [8]:
# Check if any non-walkover matches missing minutes data
data[(data["minutes"].isna()) & (data["score"] != "W/O")]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_id,winner_seed,winner_entry,winner_name,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
550,2024-0495,Dubai,Hard,32,A,2024-02-26,122330,7.0,NaN,Alexander Bublik,...,67.0,49.0,21.0,17.0,1.0,4.0,23.0,1699.0,5.0,5110.0
1277,2024-520,Roland Garros,Clay,128,G,2024-05-27,207989,3.0,NaN,Carlos Alcaraz,...,97.0,61.0,12.0,22.0,7.0,16.0,3.0,7300.0,4.0,6305.0
1278,2024-520,Roland Garros,Clay,128,G,2024-05-27,207989,3.0,NaN,Carlos Alcaraz,...,76.0,56.0,27.0,22.0,8.0,14.0,3.0,7300.0,2.0,8770.0
1279,2024-520,Roland Garros,Clay,128,G,2024-05-27,100644,4.0,NaN,Alexander Zverev,...,71.0,49.0,14.0,17.0,7.0,12.0,4.0,6305.0,7.0,4425.0
1281,2024-520,Roland Garros,Clay,128,G,2024-05-27,206173,2.0,NaN,Jannik Sinner,...,67.0,45.0,17.0,15.0,6.0,10.0,2.0,8770.0,10.0,3555.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,2024-0096,Paris Olympics,Clay,64,O,2024-07-29,207989,2.0,NaN,Carlos Alcaraz,...,46.0,33.0,12.0,10.0,3.0,5.0,3.0,8130.0,13.0,3410.0
2001,2024-0096,Paris Olympics,Clay,64,O,2024-07-29,104925,1.0,NaN,Novak Djokovic,...,38.0,22.0,10.0,9.0,8.0,13.0,2.0,8460.0,16.0,2340.0
2002,2024-0096,Paris Olympics,Clay,64,O,2024-07-29,207989,2.0,NaN,Carlos Alcaraz,...,35.0,21.0,4.0,7.0,4.0,9.0,3.0,8130.0,19.0,2115.0
2003,2024-0096,Paris Olympics,Clay,64,O,2024-07-29,104925,1.0,NaN,Novak Djokovic,...,65.0,47.0,16.0,12.0,6.0,6.0,2.0,8460.0,3.0,8130.0


In [ ]:
# Replace missing minutes data

# Exporting to Excel file to fill in missing data
df = data.loc[(data["minutes"].isna() == True) & (data["score"] != "W/O")]
df = df[["tourney_name", "winner_name", "loser_name", "tourney_id", "winner_id", "loser_id", "minutes"]]
df.to_excel("missing_minutes_template.xlsx")

In [ ]:
# NOTE: Change minutes column to fixed_minutes and delete index column and then rename to missing_minutes_final
# Import Excel file with fixed minutes data
fixes = pd.read_excel("missing_minutes_final.xlsx")

# Left merge fixed dataframe onto main dataframe
data = data.merge(fixes, on=["tourney_id", "winner_id", "loser_id"], how="left", suffixes=('', '_fixed'))

# Replace NA minutes with new fixed minutes
data["minutes"] = data["minutes"].fillna(data["fixed_minutes"])

# Delete helper column and duplicate columns
data = data.drop(columns=["fixed_minutes", "tourney_name_fixed", "winner_name_fixed", "loser_name_fixed"])

## Handle Nulls & Create Helper Columns

We fill missing categorical/ranking information and create a helper column to flag walkovers.

In [9]:
# Replace winner/loser entry nulls with "Direct" to indicate player's Direct entry into tournament

data.loc[
    data["winner_entry"].isna() == True,
    'winner_entry'
] = "Direct"

data.loc[
    data["loser_entry"].isna() == True,
    'loser_entry'
] = "Direct"

In [10]:
# Replace null loser_rank_points with 0 to indicate no ATP rank = no ATP points
data.loc[
    data["loser_rank_points"].isna() == True,
    'loser_rank_points',
] = 0

In [11]:
# Created walkover column to indicate if match was a walkover
data['walkover'] = data['minutes'].isna()

In [12]:
# Check any remaining null values
data.isna().sum()

tourney_id               0
tourney_name             0
surface                  0
draw_size                0
tourney_level            0
tourney_date             0
winner_id                0
winner_seed           1488
winner_entry             0
winner_name              0
winner_hand              0
winner_ht                1
winner_ioc               0
winner_age               0
loser_id                 0
loser_seed            2025
loser_entry              0
loser_name               0
loser_hand               0
loser_ht                 4
loser_ioc                0
loser_age                0
score                    0
best_of                  0
round                    0
minutes                211
w_ace                   22
w_df                    22
w_svpt                  22
w_1stIn                 22
w_1stWon                22
w_2ndWon                22
w_SvGms                 23
w_bpSaved               22
w_bpFaced               22
l_ace                   22
l_df                    22
l

## Missing Data Info

- winner_seed / loser_seed: Missing for unseeded players, since only a subset of players are officially seeded in tournaments.
- minutes, winner/loser match statistics: Missing for walkovers (W/O), because no points or stats are recorded.
- loser_rank / loser_rank_points: Missing for players without an ATP ranking or ATP points at the time of the match.

## Parse Match Scores

We parse the score strings into structured numeric features representing sets, games, and tiebreaks.

In [13]:
def parse_match_score(score):
    """
    Parse a tennis match score string into structured stats.
    Returns:
        w_sets_won, l_sets_won, total_sets,
        w_games_won, l_games_won, total_games, total_tiebreaks
    """
    
    if score == "W/O":
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    w_sets_won = 0
    l_sets_won = 0
    total_sets = 0
    w_games_won = 0
    l_games_won = 0
    total_games = 0
    total_tiebreaks = 0
    
    split_sets = score.split(" ")

    for set_score in split_sets:
        if set_score == "RET" or set_score == "DEF":
            break

        split_score = set_score.split("-")
        
        w_games = int(split_score[0])

        if "(" in set_score:
            total_tiebreaks += 1
            split_tiebreak = (split_score[1]).split("(")
            l_games = int(split_tiebreak[0])
        else:
            l_games = int(split_score[1])

        if w_games > l_games:
            w_sets_won += 1
        else:
            l_sets_won += 1

        total_sets += 1
        w_games_won += w_games
        l_games_won += l_games
        total_games += (w_games + l_games)

    return w_sets_won, l_sets_won, total_sets, w_games_won, l_games_won, total_games, total_tiebreaks

# Create a DataFrame from the applied function results
result_df = pd.DataFrame(
    data["score"].apply(parse_match_score).tolist(),
    columns=["w_sets_won", "l_sets_won", "total_sets", "w_games_won", "l_games_won", "total_games", "total_tiebreaks"],
    index=data.index
)

# Assign back to original DataFrame
data[["w_sets_won", "l_sets_won", "total_sets", "w_games_won", "l_games_won", "total_games", "total_tiebreaks"]] = result_df

## Export Cleaned Yearly Dataset

After cleaning the dataset for the year, we export it for consistency with future years.

In [ ]:
# CHANGES: File name to correct year
data.to_csv("data/processed/atp_matches_2016_cleaned.csv")